In [ ]:
import os
import sys
import logging

import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.pytorch import PyTorch
from botocore.exceptions import ClientError

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = "eu-central-1"
sm = boto3.Session().client(service_name="sagemaker", region_name=region)
raw_input_path = f"s3://{bucket}/imdb/processing/raw/small/raw.csv"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

## BYO Docker

In [ ]:
import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
ecr_repository = "sagemaker-pytorch-processing-container"
tag = ":latest"

uri_suffix = "amazonaws.com"
processing_repository_uri = f"{account_id}.dkr.ecr.{region}.{uri_suffix}/{ecr_repository + tag}"

### Step 1 - Set up the Experiment

Create an experiment to track all the model training iterations. Experiments are a great way to organize your data science work. You can create experiments to organize all your model development work for : [1] a business use case you are addressing (e.g. create experiment named “customer churn prediction”), or [2] a data science team that owns the experiment (e.g. create experiment named “marketing analytics experiment”), or [3] a specific data science and ML project. Think of it as a “folder” for organizing your “files”.

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

### Create an Experiment

In [ ]:
imdb_experiment = Experiment.create(
    experiment_name=f"Imdb-Experiment",
    description="Testing thing out with IMDB experiment",
    sagemaker_boto_client=sm,
)
print(imdb_experiment)

### Step 2 - Track Experiment

## Data pre-processing

In [ ]:
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(
    command=["python3"],
    image_uri=processing_repository_uri,
    role=role,
    instance_count=1,
    instance_type="ml.t3.medium",
)

Run the same `preprocessing.py` script you ran above, but now, this code is running inside of the Docker container you built in this notebook, not the scikit-learn image maintained by Amazon SageMaker. You can add the dependencies to the Docker image, and run your own pre-processing, feature-engineering, and model evaluation scripts inside of this container.

In [ ]:
script_processor.run(
    code="source/preprocessing.py",
    inputs=[
        ProcessingInput(source=raw_input_path, destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    arguments=[
        "--train-test-split-ratio",
        "0.2",
        "--model_name",
        "distilbert-base-uncased",
    ],
    experiment_config={
        "TrialComponentDisplayName": "Preprocessing",
    },
)
script_processor_job_description = script_processor.jobs[-1].describe()
output_config = script_processor_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "train":
        preprocessed_training_data = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test":
        preprocessed_test_data = output["S3Output"]["S3Uri"]

In [ ]:
# processing_job_name = script_processor_job_description['ProcessingJobName']
processing_job_name = 'sagemaker-pytorch-processing-container-2021-07-08-10-29-36-746'

In [ ]:
tracker = Tracker.load(processing_job_name=processing_job_name)
preprocessing_trial_component = tracker.trial_component

In [ ]:
from sagemaker.huggingface import HuggingFace
from sagemaker.pytorch import PyTorch

# create trial
tf_trial_name = "imdb-experiment-tf-trial"
# tf_trial = Trial.create(
#     trial_name=tf_trial_name,
#     experiment_name=imdb_experiment.experiment_name,
#     sagemaker_boto_client=sm,
# )

# associate the proprocessing trial component with the current trial
tf_trial.add_trial_component(preprocessing_trial_component)

model_path_tf = f"s3://{bucket}/imdb/model_train/tf"

hyperparameters = {
    "epochs": 1,
    "train_batch_size": 32,
    "model_name": "distilbert-base-uncased"
,
}

metric_definitions=[
    {'Name': 'traning:loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:loss', 'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:accuracy', 'Regex': "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:f1', 'Regex': "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:precision', 'Regex': "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:recall', 'Regex': "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:runtime', 'Regex': "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:samples_per_second', 'Regex': "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}]

# create the Estimator
estimator_tf = PyTorch(
    entry_point="train_tf.py",
    source_dir="source",
    instance_type='ml.g4dn.xlarge',
    instance_count=1,
    role=role,
    framework_version="1.6",
    py_version="py36",
    hyperparameters=hyperparameters,
    output_path=model_path_tf,
    metric_definitions=metric_definitions,
)

estimator_tf.fit(
    inputs={'train': preprocessed_training_data, 'test': preprocessed_test_data},
    experiment_config={
        "TrialName": tf_trial.trial_name,
        "TrialComponentDisplayName": "Training",
    },
)

In [ ]:
import json
from sagemaker.s3 import S3Downloader

script_processor.run(
    code="source/evaluation_tf.py",
    inputs=[
        ProcessingInput(source=estimator_tf.model_data, destination="/opt/ml/processing/model"),
        ProcessingInput(source=preprocessed_test_data, destination="/opt/ml/processing/test"),
    ],
    outputs=[ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation")],
    experiment_config={
        "TrialName": tf_trial.trial_name,
        "TrialComponentDisplayName": "Evaluating",
    },
)
evaluation_job_description = script_processor.jobs[-1].describe()

In [ ]:
from sagemaker.huggingface import HuggingFace
from sagemaker.pytorch import PyTorch

# create trial
svm_trial_name = "imdb-experiment-svm-trial"
# svm_trial = Trial.create(
#     trial_name=svm_trial_name,
#     experiment_name=imdb_experiment.experiment_name,
#     sagemaker_boto_client=sm,
# )

# associate the proprocessing trial component with the current trial
svm_trial.add_trial_component(preprocessing_trial_component)

model_path_svm = f"s3://{bucket}/imdb/model_train/svm"

hyperparameters = {
    "epochs": 1,
    "train_batch_size": 32,
    "model_name": "distilbert-base-uncased"
,
}

estimator_svm = PyTorch(
    entry_point="train_svm.py",
    source_dir="source",
    role=role,
    framework_version="1.6",
    py_version="py3",
    instance_type='ml.c4.xlarge',
    instance_count=1,
    output_path=model_path_svm,
)

estimator_svm.fit(
    inputs={'train': preprocessed_training_data, 'test': preprocessed_test_data},
    experiment_config={
        "TrialName": svm_trial.trial_name,
        "TrialComponentDisplayName": "Training",
    },
)